In [ ]:
pip install transformers datasets evaluate seqeval

In [ ]:
import pandas as pd
from transformers import AutoTokenizer
from datasets import load_dataset
from huggingface_hub import notebook_login
from datasets import Dataset
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
train_dataset = load_dataset("json", data_files="train.json")
test_dataset = load_dataset("json", data_files="test.json")
train_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'full_text', 'trailing_whitespace', 'tokens', 'document'],
        num_rows: 43501
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
inputs = train_dataset['train'][0]['tokens']
inputs = tokenizer(inputs, is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'a',
 'student',
 "'",
 's',
 'assessment',
 'was',
 'found',
 'on',
 'device',
 'bearing',
 'im',
 '##ei',
 ':',
 '06',
 '-',
 '1847',
 '##55',
 '-',
 '86',
 '##6',
 '##8',
 '##51',
 '-',
 '3',
 '.',
 'the',
 'document',
 'falls',
 'under',
 'the',
 'various',
 'topics',
 'discussed',
 'in',
 'our',
 'optimization',
 'curriculum',
 '.',
 'can',
 'you',
 'please',
 'collect',
 'it',
 '?',
 '[SEP]']

In [ ]:
inputs.word_ids()

[None,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 11,
 12,
 13,
 14,
 14,
 15,
 16,
 16,
 16,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 None]

In [ ]:
id2label = {
    0: "O",
    1: "B-NAME_STUDENT",
    2: "I-NAME_STUDENT",
    3: "B-EMAIL",
    4: "I-EMAIL",
    5: "B-USERNAME",
    6: "I-USERNAME",
    7: "B-ID_NUM",
    8: "I-ID_NUM",
    9: "B-PHONE_NUM",
    10: "I-PHONE_NUM",
    11: "B-URL_PERSONAL",
    12: "I-URL_PERSONAL",
    13: "B-STREET_ADDRESS",
    14: "I-STREET_ADDRESS",
}
label2id = {
    "O": 0,
    "B-NAME_STUDENT": 1,
    "I-NAME_STUDENT": 2,
    "B-EMAIL": 3,
    "I-EMAIL": 4,
    "B-USERNAME": 5,
    "I-USERNAME": 6,
    "B-ID_NUM": 7,
    "I-ID_NUM": 8,
    "B-PHONE_NUM": 9,
    "I-PHONE_NUM": 10,
    "B-URL_PERSONAL": 11,
    "I-URL_PERSONAL": 12,
    "B-STREET_ADDRESS": 13,
    "I-STREET_ADDRESS": 14,
}

In [ ]:
def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word = None
  for word_id in word_ids:
    if word_id != current_word:
      current_word = word_id
      label = -100 if word_id is None else label2id[labels[word_id]]
      new_labels.append(label)

    elif word_id is None:
      new_labels.append(-100)

    else:
      label = label2id[labels[word_id]]

      if label %2 == 1:
        label = label + 1
      new_labels.append(label)

  return new_labels



In [ ]:
labels = train_dataset['train'][0]['labels']
word_ids = inputs.word_ids()
print(labels,word_ids)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 11, 12, 13, 14, 14, 15, 16, 16, 16, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, None]


In [ ]:
align_labels_with_tokens(labels,word_ids)

[-100,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -100]

In [ ]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'],truncation=True,is_split_into_words=True)
  all_labels = examples['labels']

  new_labels = []
  for i,labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels,word_ids))

  tokenized_inputs['labels'] = new_labels
  return tokenized_inputs

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True,remove_columns=train_dataset['train'].column_names)
tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True,remove_columns=test_dataset['train'].column_names)


Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_dataset['train'][0]['labels']

[-100,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -100]

In [ ]:
##Data collation
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer)


In [ ]:
##Metrics
!pip install evaluate seqeval

In [ ]:
import evaluate
metric = evaluate.load('seqeval')

In [ ]:
label_list = set()
for example in train_dataset['train']:
    labels = example['labels']
    for label in labels:
        label_list.add(label)
label_list = list(label_list)
label_list

['O',
 'B-USERNAME',
 'B-URL_PERSONAL',
 'I-STREET_ADDRESS',
 'I-EMAIL',
 'B-PHONE_NUM',
 'I-URL_PERSONAL',
 'I-NAME_STUDENT',
 'B-STREET_ADDRESS',
 'B-NAME_STUDENT',
 'B-ID_NUM',
 'B-EMAIL',
 'I-ID_NUM',
 'I-PHONE_NUM',
 'I-USERNAME']

In [ ]:
import numpy as np
def compute_metrics(eval_preds):
  logits,labels = eval_preds
  predictions = np.argmax(logits, axis = -1)
  true_labels = [[label_list[l] for l in label if l!=-100] for label in labels]

  true_predictions = [[label_list[p] for p,l in zip(prediction, label) if l!=-100]
                      for prediction, label in zip(predictions, labels)]
  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

  return {"precision": all_metrics['overall_precision'],
          "recall": all_metrics['overall_recall'],
          "f1": all_metrics['overall_f1'],
          "accuracy": all_metrics['overall_accuracy'],
          }

In [ ]:
##Train the model
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-cased", num_labels=15, id2label=id2label, label2id=label2id)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pip install accelerate -U

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="distil_bert_pii_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset['train'],
    eval_dataset=tokenized_test_dataset['train'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.012800,0.012586,0.264263,0.454701,0.334261,0.997005
2,0.006000,0.013631,0.243459,0.499878,0.327441,0.996516


TrainOutput(global_step=5438, training_loss=0.02652406556591386, metrics={'train_runtime': 1331.8476, 'train_samples_per_second': 65.324, 'train_steps_per_second': 4.083, 'total_flos': 2216373693950286.0, 'train_loss': 0.02652406556591386, 'epoch': 2.0})

In [ ]:
from transformers import pipeline
checkpoint = "/content/distil_bert_pii_model/checkpoint-5438"
token_classifier = pipeline(
    "token-classification", model=checkpoint, aggregation_strategy="simple"
)
token_classifier("Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla Challenge & selection The tool I use to help all stakeholders finding their way through the complexity of a project is the mind map. What exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic representation technique that follows the natural functioning of the mind and allows the brain's potential to be released. Cf Annex1 This tool has many advantages: • It is accessible to all and does not require significant material investment and can be done quickly • It is scalable • It allows categorization and linking of information • It can be applied to any type of situation: notetaking, problem solving, analysis, creation of new ideas • It is suitable for all people and is easy to learn • It is fun and encourages exchanges • It makes visible the dimension of projects, opportunities, interconnections • It synthesizes • It makes the project understandable • It allows you to explore ideas The creation of a mind map starts with an idea/problem located at its center. This starting point generates ideas/work areas, incremented around this center in a radial structure, which in turn is completed with as many branches as new ideas. This tool enables creativity and logic to be mobilized, it is a map of the thoughts. Creativity is enhanced because participants feel comfortable with the method. Application & Insight I start the process of the mind map creation with the stakeholders standing around a large board (white or paper board). In the center of the board, I write and highlight the topic to design. Through a series of questions, I guide the stakeholders in modelling the mind map. I adapt the series of questions according to the topic to be addressed. In the type of questions, we can use: who, what, when, where, why, how, how much. The use of the “why” is very interesting to understand the origin. By this way, the interviewed person frees itself from paradigms and thus dares to propose new ideas / ways of functioning. I plan two hours for a workshop. Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla After modelling the mind map on paper, I propose to the participants a digital visualization of their work with the addition of color codes, images and interconnections. This second workshop also lasts two hours and allows the mind map to evolve. Once familiarized with it, the stakeholders discover the power of the tool. Then, the second workshop brings out even more ideas and constructive exchanges between the stakeholders. Around this new mind map, they have learned to work together and want to make visible the untold ideas. I now present all the projects I manage in this type of format in order to ease rapid understanding for decision-makers. These presentations are the core of my business models. The decision-makers are thus able to identify the opportunities of the projects and can take quick decisions to validate them. They find answers to their questions thank to a schematic representation. Approach What I find amazing with the facilitation of this type of workshop is the participants commitment for the project. This tool helps to give meaning. The participants appropriate the story and want to keep writing it. Then, they easily become actors or sponsors of the project. A trust relationship is built, thus facilitating the implementation of related actions. Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla Annex 1: Mind Map Shared facilities project")

[{'entity': 'B-NAME_STUDENT',
  'score': 0.99112636,
  'index': 14,
  'word': 'Nat',
  'start': 52,
  'end': 55},
 {'entity': 'I-NAME_STUDENT',
  'score': 0.9441846,
  'index': 15,
  'word': '##hali',
  'start': 55,
  'end': 59},
 {'entity': 'I-NAME_STUDENT',
  'score': 0.9808652,
  'index': 16,
  'word': '##e',
  'start': 59,
  'end': 60},
 {'entity': 'I-NAME_STUDENT',
  'score': 0.9486387,
  'index': 17,
  'word': 'S',
  'start': 61,
  'end': 62},
 {'entity': 'I-NAME_STUDENT',
  'score': 0.944961,
  'index': 18,
  'word': '##yl',
  'start': 62,
  'end': 64},
 {'entity': 'I-NAME_STUDENT',
  'score': 0.9632347,
  'index': 19,
  'word': '##la',
  'start': 64,
  'end': 66},
 {'entity': 'I-NAME_STUDENT',
  'score': 0.9683128,
  'index': 20,
  'word': 'Challenge',
  'start': 67,
  'end': 76},
 {'entity': 'I-STREET_ADDRESS',
  'score': 0.57946706,
  'index': 73,
  'word': '##e',
  'start': 294,
  'end': 295}]

In [ ]:
from transformers import create_optimizer
batch_size = 8
num_train_epochs = 3
num_train_steps = (len(tokenized_train_dataset["train"]) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [ ]:
from transformers import TFAutoModelForTokenClassification
model = TFAutoModelForTokenClassification.from_pretrained(
    "distilbert-base-cased", num_labels=15, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able t

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train_dataset["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_test_dataset["train"],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

In [ ]:
!pip install tensorflow

In [66]:
import tensorflow as tf
import tensorflow as tf.compat.v1.ConfigProto(device_count = {'GPU':1, 'CPU':10}

model.compile(optimizer=optimizer)

In [67]:
from transformers.keras_callbacks import KerasMetricCallback
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [68]:
from transformers.keras_callbacks import PushToHubCallback

In [69]:
push_to_hub_callback = PushToHubCallback(
    output_dir="distil_bert_pii_model-fine-tuned",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/distil_bert_pii_model-fine-tuned is already a clone of https://huggingface.co/Ozgunn/distil_bert_pii_model-fine-tuned. Make sure you pull the latest changes with `repo.git_pull()`.


In [70]:
callbacks = [metric_callback, push_to_hub_callback]

In [71]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

In [72]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

Epoch 1/3


Exception ignored in: <function TensorflowDatasetMixin.to_tf_dataset.<locals>.cleanup_callback at 0x7a04910ea290>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py", line 526, in cleanup_callback
    def cleanup_callback(ref):
KeyboardInterrupt: 


AssertionError: Concurrent access?